In [1]:
'''
File for loading data as a class and data processing functions
Include missing data imputation using:
    1) Ignore missing
    2) EWMA
    3) KNN
Calculate the 10-year moving averages for each explanatory variable
Prepare the input data for models
'''

import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

In [2]:
COUNTRY_LIST = sorted(list(pd.read_csv("data/countries.csv")['Countries'].values))
COL_NAME = ['Country Name', 'Country Code', '2002','2003','2004','2005','2006','2007', 
            '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 
            '2017', '2018', '2019', '2020']
FILE_NAMES = ['data/coal rent.csv', 'data/gas rent.csv', 'data/oil rent.csv', 'data/fossil fuel.csv', 'data/emission coal.csv',
              'data/emission total.csv', 'data/emission natural gas.csv', 'data/emission petroleum.csv', 'data/GDP.csv', 
              'data/energy consumption per GDP.csv']
RATE = 0.88

class Data:
    '''
    Stores each input file as a data object
    Parameters:
        file: a string that represents the file name of the input data
        data: a dataframe that stores the dataset
        moving_avg: a dataframe that stores the moving average of the dataset
    '''
    def __init__(self, file, data = None, moving_avg = None):
        self.file = file
        
    def WorldbankData(self):
        '''
        Load the dataset obtained from the world bank website
        Store the dataset in the field 'data'
        '''
        data = pd.read_csv(self.file).iloc[:, 2:]
        data.columns = COL_NAME
        self.data = data.drop(columns = ['Country Code'])
    
    def EiaData(self):
        '''
        Load the dataset obtained from the eia.gov
        Store the dataset in the field 'data'
        '''
        data = pd.read_csv(self.file, skiprows = 1)
        target_yr = COL_NAME[1:].copy()
        target_yr[0] = 'Unnamed: 1'
        data = pd.DataFrame(data, columns = target_yr).iloc[1:,:]
        data = data.rename(columns = {"Unnamed: 1": "Country Name"})
        data['Country Name'] = data['Country Name'].str.lstrip()
        self.data = data
    
    def MissingImpute(self, method):
        '''
        Impute the missing values
        Update the data field with the imputed version
        Parameter:
            method: a string that represents the imputation method the user wants to use
        '''
        temp = self.data.copy()
        temp = temp.replace('0', np.NaN)
        temp = temp.replace('..', np.NaN)
        temp = temp.replace('--', np.NaN)
        temp = temp[temp['Country Name'].isin(COUNTRY_LIST)]
        
        if (method == 'knn'):
            temp = temp.replace(np.NaN, 0).transpose()
            new_header = temp.iloc[0]
            temp = temp[1:]
            temp.columns = new_header
            
            imputer = KNNImputer(n_neighbors = 5)
            temp = pd.DataFrame(imputer.fit_transform(temp), columns = temp.columns)
            temp = temp.transpose()
            temp.reset_index(inplace = True)
            temp.columns = self.data.columns
            
        if (method == 'ewma'): 
            # EWMA(t) = lambda * x(t) + (1-lambda)EWMA(t-1)
            # EWMA(0) = mean(x)
            # The rate parameter can be tuned
            temp.iloc[:, 1:] = temp.iloc[:, 1:].astype(float)
            ewma = [0 for i in range(len(temp.columns))]
            # for each row of data, calculate the ewma values according to the formula
            for j in range(len(temp)):
                ewma[0] = temp.iloc[j, 1:].mean(skipna = True)
                for i in range(1, len(temp.columns)):
                    if (np.isnan(temp.iloc[j, i]) and i > 1):
                        ewma[i] = RATE * temp.iloc[j, i-1] + (1 - RATE) * ewma[i - 1]
                        temp.iloc[j, i] = ewma[i]
                    elif (np.isnan(temp.iloc[j, 1])):
                        ewma[i] = ewma[i - 1]
                        temp.iloc[j, i] = ewma[i]
                    else:
                        ewma[i] = RATE * temp.iloc[j, i] + (1 - RATE) * ewma[i - 1]
        
        self.data = temp
        
    def CalcMA(self):
        '''
        Calculate the 10-year moving average for the dataset
        Store the 10-yr moving average dataset in the field 'moving_avg'
        '''
        data = self.data
        df_ma = pd.DataFrame(list(data['Country Name'].values), columns = ['Country Name'])
        data.iloc[:, 1:] = data.iloc[:, 1:].astype(float)
        temp = data.iloc[:, 1:].rolling(window = 10, axis = 1, min_periods = 1).mean()
        temp.index = np.arange(0, len(temp))
        temp = temp[['2011','2012','2013','2014','2015','2016','2017', '2018','2019','2020']]
        df_ma = df_ma.merge(temp, left_index=True, right_index=True)
        self.moving_avg = df_ma
        

FileNotFoundError: [Errno 2] No such file or directory: 'data/countries.csv'

In [ ]:
def DataMovingAvg(files, impute_method):
    '''
    Create a list of dataframes, each dataframe contains the moving average data for each explanatory variable
    Parameters:
        files: a list of strings that represent the file names for the explanatory variables
        impute_method: the method of imputation that the user wants to use
        
    Return:
        data_ma: a list of dataframes
    '''
    data_ma = []
    for i in range(len(files)):
        data = Data(files[i])
        if (i <= 3):
            data.WorldbankData()
        else:
            data.EiaData()
        data.MissingImpute(impute_method)
        data.CalcMA()
        data_ma.append(data.moving_avg)
    return data_ma     
            

In [ ]:
def OrganizeDf(time_range, data_ma):
    '''
    Organize the list of dataframes obtained from DataMovingAvg into a list of 10 dataframes, 
    each dataframe represents the data for one year, and is in the format of:
        row: each country
        column: the 10-yr moving average data for each explanatory variables
        
    Parameters:
        time_range: a list of strings that specifies the years that the output list will include
                    for example, the format will be ['2011', '2012', ...]
        data_ma: a list of dataframes that needs to be organized
        
    Output:
        df_list: a list of dataframes 
    '''
    df_list = []
    for i in range(len(time_range)):
        df = pd.DataFrame(COUNTRY_LIST, columns = ['Country Name'])
        for data in data_ma:
            specific_yr = data[['Country Name',time_range[i]]]
            df = df.merge(specific_yr, on = 'Country Name', how='left')  #left join; total 174 countries/rows
    
        df.columns = ['Country Name','coal_rent_ma','gas_rent_ma','oil_rent_ma', 'coal_emission_ma', 'total_emission_ma',
                      'gas_emission_ma', 'petr_emission_ma', 'gdp_ma', 'energy_gdp_ma','fossil_fuel_ma']
        #set country name as index
        df.set_index('Country Name', inplace = True)
        #convert them into gdp per capita
        #remind:if the country selected cold not find population, we assume all values in this country are 1
        for country_name in df.index:
            if sum(population.moving_avg['Country Name'] == country_name) == 1:
                df.loc[country_name] = df.loc[country_name] / population.moving_avg[population.moving_avg['Country Name'] == country_name][time_range[i]].values.astype(int)[0] * 1000000
                #multiply 1000000 to make the values look bigger 
            else:
                df.loc[country_name] = 1

        #combine three columns below
        df["total_rent_ma"] = df['coal_rent_ma'] + df['gas_rent_ma'] + df['oil_rent_ma']
        df.drop(['coal_rent_ma','gas_rent_ma','oil_rent_ma'],axis=1,inplace=True)
        
        df_list.append(df)
    return df_list
    

# Example

In [4]:
# Read data from the file (must calculate the population moving avg separately)
population = Data('natural gas emission.csv')
population.EiaData()
population.data

NameError: name 'Data' is not defined

In [3]:
# Impute missing data using the selected method
population.MissingImpute('ewma')
population.data

NameError: name 'population' is not defined

In [ ]:
# Calculate 10-year moving average of the dataset
population.CalcMA()
population.moving_avg

,Country Name,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Albania,3038.52600,3017.31700,2995.89400,2975.0360,2955.4070,2937.74800,2922.76600,2910.70300,2901.3910,2894.583493
1,Algeria,34047.25000,34600.13000,35187.72000,35810.8700,36468.7000,37159.74000,37881.96000,38629.88000,39399.5300,40103.486843
2,Angola,20667.47000,21426.32000,22215.75000,23034.1200,23879.2000,24748.46000,25639.60000,26553.37000,27486.9200,28334.617683
3,Antigua and Barbuda,83.54542,84.75116,85.95862,87.1678,88.3787,89.54538,90.66784,91.74608,92.7801,93.681334
4,Argentina,39542.45000,39950.48000,40362.24600,40777.6820,41196.3170,41618.08700,42043.15900,42471.09300,42900.8110,43288.497034
...,...,...,...,...,...,...,...,...,...,...,...
168,Vanuatu,217.39530,222.95310,228.72080,234.6973,240.8819,247.27660,253.88010,260.68170,267.6506,274.010491
169,Venezuela,27017.34200,27443.38400,27866.33000,28270.9700,28635.9070,28936.01700,29151.50500,29276.63600,29325.1210,29333.346552
170,Vietnam,85094.33000,85921.08000,86766.18000,87631.2900,88515.7300,89418.02000,90336.12000,91263.11000,92196.4800,93040.805507
171,Zambia,12400.45000,12749.80000,13116.78000,13501.7000,13904.0200,14323.02000,14758.08000,15209.60000,15675.4700,16101.415781


In [ ]:
# Create a list of dataframes
# Each dataframe contains the moving average data for one explanatory variable
data_ma_list = DataMovingAvg(FILE_NAMES, 'ewma')
data_ma_list

[            Country Name      2011      2012      2013      2014      2015  \
 0                Albania  0.003504  0.003504  0.003403  0.002674  0.002939   
 1                Algeria       NaN       NaN       NaN       NaN       NaN   
 2                 Angola       NaN       NaN       NaN       NaN       NaN   
 3    Antigua and Barbuda       NaN       NaN       NaN       NaN       NaN   
 4              Argentina  0.000792  0.000792  0.000826  0.000780  0.000773   
 ..                   ...       ...       ...       ...       ...       ...   
 152           Uzbekistan  0.192223  0.194993  0.194790  0.166736  0.149922   
 153              Vanuatu       NaN       NaN       NaN       NaN       NaN   
 154              Vietnam  1.215732  1.296901  1.340985  1.233536  1.163864   
 155               Zambia  0.026254  0.022793  0.018577  0.012934  0.009167   
 156             Zimbabwe  2.232021  2.236884  2.209496  2.084632  1.905205   
 
          2016      2017      2018      2019      

In [ ]:
'''
Organize the datasets into the format of: 
    row of each dataframe is countries
    column of each dataframe is explanatory variables
'''
input_yearly = OrganizeDf(COL_NAME[-10:-1], data_ma_list)
input_yearly

[                     coal_emission_ma  total_emission_ma  gas_emission_ma  \
 Country Name                                                                
 Albania                  21113.084562          65.869727      1292.585140   
 Algeria                   2930.300635          33.946410      3781.362866   
 Angola                    1700.944389           0.000000       991.634784   
 Antigua and Barbuda               NaN           0.000000      8675.838422   
 Argentina                 2242.415196          90.474090      5028.418984   
 ...                               ...                ...              ...   
 Vanuatu                           NaN           0.000000       805.420364   
 Venezuela                         NaN          20.740862      5252.522512   
 Vietnam                    726.519067         539.449007      2083.583197   
 Zambia                     695.675999          21.148690       321.304206   
 Zimbabwe                  2279.354424         573.451389       